# Forest management problem

In [36]:
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

from desdeo_problem.problem import DiscreteDataProblem
from desdeo_tools.scalarization import SimpleASF, DiscreteScalarizer, StomASF, PointMethodASF
from desdeo_tools.solver import DiscreteMinimizer
from shapley_values.explanations import *
from shapley_values.utilities import *
from IPython.core.display import display, HTML

# parameters
## the ASF used
ASF = PointMethodASF

# Load and setup problem (all objectives are defined to be maximized, so we multiply the whole dataframe by -1)
df = -1*pd.read_csv("../data/forest.csv")
pareto_front = df.to_numpy()

# compute the ideal and nadir
ideal = np.min(pareto_front[:, 0:3], axis=0)
nadir = np.max(pareto_front[:, 0:3], axis=0)

objective_names = ["Income", "Carbon", "Habitat index"]
n_objectives = 3

# we do not know the original decision variables, so we use a single 'dummy' variable just to make sure DiscreteDataProblem works correctly with the data
problem = DiscreteDataProblem(df, "dummy", objective_names, nadir, ideal)

asf = ASF(nadir, ideal)

# sample the Pareto front for missing data to be used while computing SHAP values
missing_data = shap.sample(pareto_front[:, 0:n_objectives], nsamples=200)

# generate the method to be used, or the black-box
bb = generate_black_box(problem, asf)

# define the explainer that computed SHAP values (use kernel SHAP)
explainer = shap.KernelExplainer(bb, missing_data)

def plot(result, objective_names=objective_names, ymin=0, ymax=1):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.set_ylim(ymin, ymax)
    ax.bar(objective_names, np.squeeze(result)) 
    plt.grid()
    plt.show()

Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [35]:
df

,Income,Carbon,Habitat index,dummy
0,-4.107322e+07,-8.145972e+09,-28086.252430,-0.0
1,-4.072116e+07,-8.131148e+09,-28417.766144,-0.0
2,-4.030586e+07,-8.113703e+09,-28751.763921,-0.0
3,-3.984705e+07,-8.094358e+09,-29094.496953,-0.0
4,-3.932061e+07,-8.072229e+09,-29445.502566,-0.0
...,...,...,...,...
1723,-6.285491e+07,-6.734194e+09,-21565.769627,-0.0
1724,-6.285491e+07,-6.734194e+09,-21565.773676,-0.0
1725,-6.285491e+07,-6.734194e+09,-21565.773676,-0.0
1726,-6.285491e+07,-6.734194e+09,-21565.773676,-0.0


In [30]:
nadir

array([4.10732250e+07, 8.14597175e+09, 2.87517639e+04, 0.00000000e+00])